In [9]:
import sys
sys.path.append('../')

%load_ext autoreload
%autoreload 2

import os
from custom_datasets.rollout_push_any import RolloutPushAnyDataset
from custom_datasets.pusht import PushTDataset
from concurrent.futures import ThreadPoolExecutor, wait
from datasets import Dataset

import re
from pathlib import Path
import zarr
import numpy as np
import torch
import torch.nn.functional as F
import imageio
import einops
from IPython.display import Video

from typing import Dict
import time

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [17]:
root_path = '/home/sm/Datasets/dynamo/datasets/dynamo_repro_datasets'
pusht_path = root_path + '/pusht_dataset'
new_pusht_path = root_path + '/pusht_dataset_ds'

vid_dir = Path(pusht_path) / "obses"
new_vid_dir = Path(new_pusht_path) / "obses"

In [6]:
pusht_dataset = PushTDataset(pusht_path)
print(len(pusht_dataset))

206


In [20]:
def load_frames(episode_idx):
    return torch.load(str(vid_dir / f"episode_{episode_idx:03d}.pth"))

def rescale_frames(t):
    t = einops.rearrange(t, "T H W C -> T C H W")
    t = F.interpolate(t, size=[96, 96], mode='bilinear', align_corners=False)
    t = F.interpolate(t, size=[224, 224], mode='bilinear', align_corners=False)
    t = einops.rearrange(t, "T C H W -> T H W C")
    return t

def save_frames(t, episode_idx):
    assert t.size(3) == 3
    print(episode_idx)
    torch.save(t, new_vid_dir / f"episode_{episode_idx:03d}.pth")

In [21]:
ot = load_frames(0)
print(ot.size())
rt = rescale_frames(ot)
print(rt.size())

torch.Size([161, 224, 224, 3])
torch.Size([161, 224, 224, 3])


In [23]:
frames = [f for f in rt.numpy()]
Path("outputs/examples/1_load_lerobot_dataset").mkdir(parents=True, exist_ok=True)
video_path = f"outputs/examples/1_load_lerobot_dataset/r.mp4"
imageio.mimsave(video_path, frames, fps=10)

Video(video_path, embed=True, width=640, height=360)

In [24]:
for episode_idx in range(len(pusht_dataset)):
    ot = load_frames(episode_idx)
    rt = rescale_frames(ot)
    save_frames(rt, episode_idx)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205


In [30]:
new_pusht_dataset = PushTDataset(new_pusht_path)

In [28]:
def show_video(episode_index, dataset):
    frames = dataset[episode_index][0].squeeze(1)
    
    frames = [(frame * 255).type(torch.uint8).permute(1, 2, 0) for frame in frames]
    frames = [frame.numpy() for frame in frames]
    Path("outputs/examples/1_load_lerobot_dataset").mkdir(parents=True, exist_ok=True)
    video_path = f"outputs/examples/1_load_lerobot_dataset/{type(dataset).__name__}.mp4"
    imageio.mimsave(video_path, frames, fps=10)
    
    return video_path

In [29]:
video_path = show_video(1, new_pusht_dataset)
Video(video_path, embed=True, width=640, height=360)

In [31]:
video_path = show_video(1, pusht_dataset)
Video(video_path, embed=True, width=640, height=360)